In [ ]:
%pip install pyspark findspark hadoop-azure azure-storage pymongo[srv]

In [13]:
from pyspark.sql import SparkSession
from pymongo import MongoClient
import os

# Configurações do Azure Data Lake
account_name = os.environ["AZURE_STORAGE_ACCOUNT_NAME"]
sas_token = os.environ["AZURE_STORAGE_SAS_TOKEN"]

# Configurações do MongoDB
mongodb_uri = os.environ["MONGODB_CONNECTION_STRING"]
mongodb_database = os.environ["MONGODB_DATABASE"]

# Inicialização do SparkSession
spark = SparkSession.builder \
    .appName("Extrair MongoDB para Azure") \
    .config("spark.jars.packages", "org.apache.hadoop:hadoop-azure:3.3.4,org.mongodb.spark:mongo-spark-connector_2.12:10.4.0") \
    .config("spark.hadoop.fs.azure", "org.apache.hadoop.fs.azure.NativeAzureFileSystem") \
    .config("spark.driver.extraJavaOptions", "-Dlog4j.configuration=file:/path/to/log4j.properties") \
    .getOrCreate()

# Conexão ao MongoDB para listar coleções
client = MongoClient(mongodb_uri)
db = client[mongodb_database]
collections = db.list_collection_names()

# Caminho do contêiner
bronze_container = f"abfs://bronze@{account_name}.dfs.core.windows.net"

# Processar as coleções do MongoDB
for collection in collections:
    print(f"Lendo coleção: {collection}")
    try:
        # Leitura da coleção do MongoDB
        df = spark.read \
            .format("mongodb") \
            .option("uri", mongodb_uri) \
            .option("database", mongodb_database) \
            .option("collection", collection) \
            .load()

        # Exibir o esquema para verificar se há dados
        df.printSchema()

        # Verificar se o DataFrame está vazio
        if df.count() == 0:
            print(f"A coleção {collection} está vazia.")
        else:
            # Caminho de saída para o Data Lake
            output_path = f"{bronze_container}/{collection}.parquet"

            # Gravar no Data Lake
            df.write.mode("overwrite").parquet(output_path)

            print(f"Coleção {collection} salva em {output_path}")
    except Exception as e:
        print(f"Erro ao processar a coleção {collection}: {e}")

# Finalizar SparkSession
spark.stop()


Lendo coleção: cars
root

A coleção cars está vazia.
Lendo coleção: sales
root

A coleção sales está vazia.
Lendo coleção: services
root

A coleção services está vazia.
Lendo coleção: employees
root

A coleção employees está vazia.
Lendo coleção: payments
root

A coleção payments está vazia.
Lendo coleção: clients
root

A coleção clients está vazia.
